# Train a Plane Detection Model from Voxel51 Dataset
This notebook trains a plane detection model using transfer learning. 
Depending on the label used, it can just detect a plane or it can try to detect the model of the plane.
A pre-trained model is used as a starting point. This means that fewer example images are needed and the training process is faster.

Images are exported from a Voxel51 Dataset into TensorFlow Records.The examples in the TFRecord are based on a selected Field from the Samples in the Voxel51 dataset. The V51 Sample field you choose should have 1 or more "detections", which are bounding boxes with a label.

From: https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD#scrollTo=wHfsJ5nWLWh9&uniqifier=1

Good stuff here too: https://www.inovex.de/blog/deep-learning-mobile-tensorflow-lite/ 

## Configure the Training

In [ ]:
training_name="mobilenet_plane_detect" # The name for the model. All of the different directories will be based on this
label_field = "planebox"  # The field from the V51 Samples around which will be used for the Labels for training.
dataset_name = "plane-dataset" # The name of the V51 dataset that will be used


# Available Model Configs (You can add more from the TF2 Model Zoo)
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 24
    },
    'ssd_mobilenet_v2_fpnlite': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 18
    },
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 18
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 18
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 18
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 18
    }
}

# change chosen model to deploy different models 
chosen_model = 'ssd_mobilenet_v2'#'efficientdet-d0'

num_steps = 40000 # The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 # Perform evaluation after so many steps



In [ ]:
# The different directories and filenames to use
train_record_fname = "/tf/dataset-export/" + training_name + "/train/tf.records"
val_record_fname = "/tf/dataset-export/" + training_name + "/val/tf.records"
val_export_dir = "/tf/dataset-export/" + training_name + "/val/"
train_export_dir = "/tf/dataset-export/" + training_name + "/train/"
model_export_dir = "/tf/model-export/" + training_name +"/"

label_map_file = "/tf/dataset-export/" + training_name + "/label_map.pbtxt"

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

pipeline_fname = '/tf/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/tf/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'
pipeline_file = '/tf/models/research/deploy/pipeline_file.config'
model_dir = '/tf/training/'+training_name+'/'

In [ ]:
# Install the different packages needed
! apt install -y protobuf-compiler libgl1-mesa-glx wget

## Download and Install TF Models
The TF Object Detection API is available here: https://github.com/tensorflow/models

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models /tf/models

In [ ]:
%%bash
cd /tf/models/research
ls
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
from google.protobuf import text_format
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

## Export the Training and Val Dataset from Voxel 51

In [ ]:
import fiftyone as fo
import math
dataset = fo.load_dataset(dataset_name)


### Explore the dataset content
Here are some basic stats on the Voxel51 dataset you are going to build training the model on. 
An example of the samples is also printed out. In the Sample, make sure the *label_field* you selected has some detections in it.

In [ ]:
print("\t\tDataset\n-----------------------------------")
view = dataset.exists(label_field).shuffle(seed=51) # You can add additional things to the query to further refine it. eg .match_tags("good_box")
print(view)
print("\n\n\tExample Sample\n-----------------------------------")
print(view.first())


### Export the dataset into TFRecords
The selected dataset samples will be exported to TensorFlow Records (TFRecords). They will be split between Training and Validation. The ratio can be adjusted below. You only need to do this once to build the dataset. If you run this a second time with the same **model_name** additional samples will be appended to the end.

In [ ]:
# The Dataset or DatasetView to export
sample_len = len(view)
val_len = math.floor(sample_len * 0.2)
train_len = math.floor(sample_len * 0.8)
print("Total: {} Val: {} Train: {}".format(sample_len,val_len,train_len))
val_view = view.take(val_len)
train_view = view.skip(val_len).take(train_len)
# Export the dataset
val_view.export(
    export_dir=val_export_dir,
    dataset_type=fo.types.TFObjectDetectionDataset,#fo.types.COCODetectionDataset,#fo.types.TFObjectDetectionDataset,
    label_field=label_field,
)

train_view.export(
    export_dir=train_export_dir,
    dataset_type=fo.types.TFObjectDetectionDataset,#fo.types.COCODetectionDataset,#fo.types.TFObjectDetectionDataset,
    label_field=label_field,
)

## Create a file with the Labels for the objects
The TF2 Object Detection API looks for a map of the labels used and a corresponding Id. You can build a list of the unique classnames by itterating the dataset. You can also just hardcode it if there only a few.

In [ ]:
def convert_classes(classes, start=1):
    msg = StringIntLabelMap()
    for id, name in enumerate(classes, start=start):
        msg.item.append(StringIntLabelMapItem(id=id, name=name))

    text = str(text_format.MessageToBytes(msg, as_utf8=True), 'utf-8')
    return text

In [ ]:
# If labelfield is a classification
class_names=[]
for sample in view.select_fields(label_field):
    if sample[label_field].label not in class_names:
        class_names.append(sample[label_field].label)
print(class_names)

In [ ]:
# If labelfield is detections
class_names=[]
for sample in view.select_fields(label_field):
    for detection in sample[label_field].detections:
        label = detection["label"]
        if label not in class_names:
            class_names.append(label)
print(class_names)

In [ ]:
# You can hard wire it too
class_names=["plane"]

In [ ]:

txt = convert_classes(class_names)
print(txt)
with open(label_map_file, 'w') as f:
        f.write(txt)

## Download a pretrained Model & default Config
A list of the models can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

The configs are here: https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/

In [ ]:
#download pretrained weights
%mkdir /tf/models/research/deploy/
%cd /tf/models/research/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

In [ ]:
#download base training configuration file
%cd /tf/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

In [ ]:
!python /tf/models/research/object_detection/dataset_tools/create_pet_tf_record.py \
    --label_map_path="/tf/dataset-export/pet/pet_label_map.pbtxt" \
    --data_dir="/tf/dataset-export/pet/" \
    --output_dir="/tf/dataset-export/pet/"

## Build the Config for training
The default config for the model being trained needs to be updated with the correct parameters and paths to the data. This just adds some standard settings, you may need to do some additional tuning if the training is not working well.

In [ ]:
# Gets the total number of classes from the Label Map

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_file)
print("working with {} classes".format(num_classes))

You may need to adjust the learning rate section below. The number used here are from the EfficentDet config. I noticed that this learning rate worked well for the small bounding boxes I was using when planes were at a high altitude. You can try increasing it if the planes take up more of the image. If the initial loss rates are high (>0) that is a probably a sign that you should adjust the Learning Rate.

You may also want to look at other aspects of the config file. They set the parameters for the model training and may need to be adjusted based on the Model Architecture you are using and the images you are training on.

In [ ]:
# write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd /tf/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_file), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set learning_rate_base in learning_rate, sane default
    s = re.sub('learning_rate_base: [.0-9]+',
               'learning_rate_base: {}'.format("8e-2"), s)
    
    # Set warmup_learning_rate in learning_rate, sane default
    s = re.sub('warmup_learning_rate: [.0-9]+',
               'warmup_learning_rate: {}'.format(.001), s)
    
    # Set warmup_steps in learning_rate, sane default
    s = re.sub('warmup_steps: [.0-9]+',
               'warmup_steps: {}'.format(2500), s)
    
    # Set total_steps in learning_rate, num_steps
    s = re.sub('total_steps: [0-9]+',
               'total_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    s = re.sub('random_scale_crop_and_pad_to_square {\s+output_size: 896\s+scale_min: 0.1\s+scale_max: 2.0\s+}',
               'random_crop_image {\n\tmin_object_covered: 1.0\n\tmin_aspect_ratio: 0.75\n\tmax_aspect_ratio: 1.5\n\tmin_area: 0.25\n\tmax_area: 0.875\n\toverlap_thresh: 0.5\n\trandom_coef: 0.125\n}',s, flags=re.MULTILINE)
    
    s = re.sub('ssd_random_crop {\s+}',
               'random_crop_image {\n\tmin_object_covered: 1.0\n\tmin_aspect_ratio: 0.75\n\tmax_aspect_ratio: 1.5\n\tmin_area: 0.10\n\tmax_area: 0.75\n\toverlap_thresh: 0.5\n\trandom_coef: 0.125\n}',s, flags=re.MULTILINE)
    
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [ ]:
%cat /tf/models/research/deploy/pipeline_file.config

# Train Custom TF2 Object Detector

This step will launch the TF2 Object Detection training. It can take a while to start-up. 
If you get an error about not finding the GPU, try shutting down the Jupyter kernel and restarting it.
While it is running, it should print out the Current Loss and which Step it is on.

* pipeline_file: defined above in writing custom training configuration
* model_dir: the location tensorboard logs and saved model checkpoints will save to
* num_train_steps: how long to train for
* num_eval_steps: perform eval on validation set after this many steps

In [ ]:
!python /tf/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps} 

# Evaluate trained model
After the model has finished training, try running it against some data to see if it atleast works.

In [ ]:

import matplotlib
import matplotlib.pyplot as plt

import io, os, glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
%ls {model_dir}

## Load model from a training checkpoint
Select a checkpoint index from above

In [ ]:
# generally you want to put the last ckpt index from training in here
checkpoint_index=41

# recover our saved model
pipeline_config = pipeline_file

checkpoint = model_dir + "ckpt-" + str(checkpoint_index)
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(checkpoint)).expect_partial()


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
# map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random

TEST_IMAGE_PATHS = glob.glob('/tf/testing/Airbus A321-271NX/*.jpg') #/tf/dataset-export/pet/images/keeshond_171.jpg') #'/tf/testing/Dassault Aviation FALCON 2000/*.jpg')
image_path = random.choice(TEST_IMAGE_PATHS)
image_np = load_image_into_numpy_array(image_path)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

detections, predictions_dict, shapes = detect_fn(input_tensor)


print(detections['detection_scores'])
label_id_offset = 1 # Depending on whether your LabelMap starts at 0 or 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.2,
      agnostic_mode=False,
)

plt.figure(figsize=(20,25))
plt.imshow(image_np_with_detections)
plt.show()

# Export the model
When you have a working model, use the TF2 Object Detection API to export it to a saved model.

In [ ]:
!python /tf/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --trained_checkpoint_dir={model_dir} \
    --pipeline_config_path={pipeline_file} \
    --output_directory {model_export_dir}

### Export a TFLite compatible model
Remeber that only Detection models that use SSDs are supported

In [ ]:
!python /tf/models/research/object_detection/export_tflite_graph_tf2.py \
  --pipeline_config_path={pipeline_file} \
  --trained_checkpoint_dir={model_dir} \
  --output_directory={model_export_dir}tflite-compatible


In [ ]:
! tflite_convert \
    --saved_model_dir="{model_export_dir}tflite/saved_model" \
    --output_file="{model_export_dir}output.tflite"

In [ ]:
#https://github.com/tensorflow/models/issues/9033#issuecomment-706573546
import cv2
import glob
import numpy as np

train_images = []

def representative_data_gen():
    path = '/tf/testing/Airbus A319-115'

    dataset_list = tf.data.Dataset.list_files(path + '/*.jpg')
    for i in range(100):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [300, 300])
        image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        yield [image]


converter = tf.lite.TFLiteConverter.from_saved_model(model_export_dir+"tflite-compatible/saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
                                       tf.lite.OpsSet.TFLITE_BUILTINS]
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]
#converter.inference_input_type = tf.int8
#converter.inference_output_type = tf.int8
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
#converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(model_export_dir+'model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | tee /etc/apt/sources.list.d/coral-edgetpu.list
!apt-get update
!apt-get -y install edgetpu-compiler

In [ ]:
!edgetpu_compiler -s {model_export_dir}model.tflite -o {model_export_dir}

### Export a TensorJS compatible model
From: https://www.tensorflow.org/js/tutorials/conversion/import_saved_model

In [ ]:
!pip install tensorflowjs

In [ ]:
! tensorflowjs_converter \
    --input_format=tf_saved_model \
    {model_export_dir}saved_model \
    {model_export_dir}web_model

In [ ]:
!saved_model_cli show --dir /tf/models/research/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model --all